In [1]:
import pandas as pd
import xlwings as xw
import openpyxl as op
import numpy as np
import datetime as dt
import numpy as np
import os
import math

In [2]:
#Muito Importante
Teams={'Prem':20,'LL':20,'L':20,'SA':20,'LP':18,'Bund':18,'Ered':18}

In [3]:
pd.set_option('display.max_rows',2000)
pd.set_option('display.max_columns',1000)

# Funções auxiliares
Algumes vêm no python Notebook

`find_data()` É importante, usa o facto dos ficheiro com os dados das ligas acabarem com 3 '_'  para encontrar os seus índices

In [4]:
def find_data(os_path):
    lista=os.listdir(os_path)
    data_files=[] #devolve o nome das pastas que têm os dados
    for i in range(len(lista)):
        if lista[i][-3:]=='___':
            data_files+=[lista[i]]
    print(data_files)
    return data_files

In [5]:
def fileyear_to_ind(year): #damos o ano e devolve o índice do ficheiro
    #lembrando que 2002 é o primeiro ano (0)
    #contamos o ano em que acaba a época
    return (year-2002)

`round_down` arredonda (para os números decimais que quisermos) para baixo

In [6]:
def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

In [7]:
round_down(1.39,1)

1.3

In [8]:
def check_wrongdate(file,sheet=0):
    #file=pd.read_excel(path+'.xlsx',sheet)
    limit=len(file.index)
    over_month=0
    month2=0
    counter=0
    counter_pd=0
    for i in range(1,limit): #ao começar no 1, temos sempre um valor anterior para comparar, se começasse no zero, comparava com o último valor [-1]
        day_=file.iloc[i]['Date']
        day2=file.iloc[i-1]['Date']
        if isinstance(day_,dt.datetime):
            counter+=1
            if isinstance(day2,str):
                if len(day2)==10:
                    date_str=dt.datetime.strptime(day2,"%d/%m/%Y")
                elif len(day2)==8:                                        #serve para datas qnão têm '20' (2/3/15)
                    day2=day2[:6]+"20"+day2[6:]
                    date_str=dt.datetime.strptime(day2,"%d/%m/%Y")
                else: 
                    print("Oops! Algo está a falhar na função date, qnd a data é uma string!")
                if ((day_.month)-(date_str.month)!=1) and ((day_.month)-(date_str.month)!=0):
                    over_month=1 #qnd muda de str para datetime, e estão trocados mes e dia a diferença entre os meses é estranha (diferente de 1)
            elif isinstance(day2,dt.datetime):
                if (day2.day==day_.day) and (day2.month!=day_.month):
                    #este if é para 'apanhar' qnd troca mes com dia, então o dia é igual, mas o mês muda em linhas consecutivas
                    month2=1
        elif isinstance(day_,pd.Timestamp):
            counter_pd+=1
    if (over_month+month2)==2:
        return (True, 
               str(counter)+' Datas em Datetime',
               str(counter_pd)+' Datas em Timestamp')
    else:
        return (False, 
               str(counter)+' Datas em Datetime',
               str(counter_pd)+' Datas em Timestamp',
               'over_month = '+str(over_month),
               'month2 = '+str(month2))

def swap_datetime(date):
    return dt.datetime.strptime((dt.datetime.strftime(date,'%Y/%d/%m')),'%Y/%m/%d')

def all_datetime(path,sheet=0,write=True): #se write=True, escreve o ficheiro nos Documentos do PC
    #false, dá-nos apenas o ficheiro já tratado aqui no python
    file=pd.read_excel(path+'.xlsx',sheet)
    limit=len(file.index)
    for it in range(0,limit):    
        if isinstance(file.iloc[it]['Date'],dt.datetime):
            file.at[it,'Date']=swap_datetime(file.iloc[it]['Date'])
        elif isinstance(file.iloc[it]['Date'],str):
            d=file.iloc[it]['Date']
            if len(d)==8:
                d=d[:6]+"20"+d[6:]
            file.at[it,'Date']=dt.datetime.strptime(d,"%d/%m/%Y")
    if write:
        with pd.ExcelWriter(path+".xlsx") as writer:
            file.to_excel(writer, 'Sheet1', index=None)
            writer.save()
    else:
        return file

In [9]:
def findall_clubs(path,teams):
    import pandas as pd
    file=pd.read_excel(path+'.xlsx')
    lista_c=[]
    counter=0
    i=0
    done=False
    while i<len(file.index) and not(done):
        if i==teams+50:
            print('Está a ver demasiadas linhas, tem erro')
        if file.iloc[i]['HomeTeam'] in lista_c:
            pass
        else:
            lista_c+=[file.iloc[i]['HomeTeam']]
            counter+=1
        if file.iloc[i]['AwayTeam'] in lista_c:
            pass
        else:
            lista_c+=[file.iloc[i]['AwayTeam']]
            counter+=1
        if counter==teams:
            done=True
        i+=1
    return lista_c 

In [10]:
def findall_clubs2(path): #menos eficiente, mas não precisa do número das equipas
    import pandas as pd
    import xlwings
    #xwfile=xw.Book(path+".xlsx") #assim não dá permission error
    #xwfile.close()
    file=pd.read_excel(path+'.xlsx')
    lista_c=[]
    for i in range(0,30):
        H=file.iloc[i]['HomeTeam']
        A=file.iloc[i]['AwayTeam']
        if H not in lista_c:
            lista_c+=[H]
        if A not in lista_c:
            lista_c+=[A]
            
    return lista_c
    
    

In [11]:
len(findall_clubs2('PLTeste'))

20

# ALPHA Functions

In [12]:
def lastgames_stats(df,clubs,g,diff=True): #ver no caderno como calcular last games stats
    for club in clubs:
        vars()[club]=[[],0]
    #df=pd.read_excel(file+'.xlsx',0)
    colunaL=[]
    colunaH=[]
    special=[]
    size=len(df.index)
    for i in range(size):
        if df.iloc[i]['ODDH_Aver.']<=df.iloc[i]['ODDA_Aver.']: #odd + baixa fora
            L=df.iloc[i]['HomeTeam']
            H=df.iloc[i]['AwayTeam']
            LO=df.iloc[i]['ODDH_Aver.']
            HO=df.iloc[i]['ODDA_Aver.']
            Diff=(df.iloc[i]['FTHG']-df.iloc[i]['FTAG'])
        else:
            H=df.iloc[i]['HomeTeam']
            L=df.iloc[i]['AwayTeam']
            HO=df.iloc[i]['ODDH_Aver.']
            LO=df.iloc[i]['ODDA_Aver.']
            Diff=(df.iloc[i]['FTAG']-df.iloc[i]['FTHG'])
            
        if vars()[L][1]<g:
            colunaL+=[0]
        elif vars()[L][1]>=g:
            colunaL+=[sum(vars()[L][0][-g:])]

        if vars()[H][1]<5:
            colunaH+=[0]
        elif vars()[H][1]>=g:
            colunaH+=[sum(vars()[H][0][-g:])]
            
        if vars()[H][1]>=g and vars()[L][1]>=g:
            special+=[int(round(sum(vars()[L][0][-g:])-sum(vars()[H][0][-g:])))]
        else:
            special+=[0]
            
        if Diff>0:
            vars()[L][0]+=[Diff*(LO/HO)]
            vars()[H][0]+=[(-Diff)*(LO/HO)]
        elif Diff==0:
            vars()[L][0]+=[0]
            vars()[H][0]+=[0]
        else:
            vars()[L][0]+=[Diff*(LO/HO)]
            vars()[H][0]+=[(-Diff)/(LO/HO)]


        vars()[L][1]+=1
        vars()[H][1]+=1
    
    if diff:
        output=pd.DataFrame({'LG_Stats_Diff':special})
    else:
        print(diff,'why')
        output=pd.DataFrame({'LG_Stats_L':colunaL,'LG_Stats_H':colunaH,'LG_Stats_Diff':special})
    
    return output
    
    
        
        

In [13]:
def low_draw_high(df): #dá as colunas das LowOdd,DrawOdd,HighOdd e se a equipa joga em casa ou não
    #ATENÇÃO
    #esta função só faz arredondamento das LowOdds, o resto fica por fazer noutra função
    #assim escolhemos o tipo de arredondamento por Odd
    #df=pd.read_excel(file+'.xlsx',0)
    size=len(df.index)
    low=[]
    draw=[]
    high=[]
    L=[] #lista de LOw clubs
    H=[] #lista de High clubs
    h_a=[] #casa(10)/fora(0)
    for i in range(size):
        draw+=[df.iloc[i]['ODDD_Aver.']]
        if df.iloc[i]['ODDH_Aver.']<df.iloc[i]['ODDA_Aver.']:
            low+=[round_down(df.iloc[i]['ODDH_Aver.'],1)]
            high+=[df.iloc[i]['ODDA_Aver.']]
            h_a+=[10]
            L+=[df.iloc[i]['HomeTeam']]
            H+=[df.iloc[i]['AwayTeam']]
        else:
            low+=[round_down(df.iloc[i]['ODDA_Aver.'],1)]
            high+=[df.iloc[i]['ODDH_Aver.']]
            h_a+=[0]
            L+=[df.iloc[i]['AwayTeam']]
            H+=[df.iloc[i]['HomeTeam']]
    output=pd.DataFrame({'Div':df['Div'],'Date':df['Date'],'LO_Team':L,'HO_Team':H,'LowOdd':low,'DrawOdd':draw,'HighOdd':high})
    return output

In [14]:
def points_made(df,clubs,x,diff=True):
    for club in clubs:
        vars()[club]=[[],0]
    l_points=[]
    h_points=[]
    points_diff=[]
    #df=pd.read_excel(file+'.xlsx',0)
    size=len(df.index)
    for i in range(size):
        res=df.iloc[i]['FTR']
        LW=False #low win
        D=False #draw
        LL=False #low lost
        if df.iloc[i]['ODDH_Aver.']<df.iloc[i]['ODDA_Aver.']:
            L=df.iloc[i]['HomeTeam']
            H=df.iloc[i]['AwayTeam']
            if res=='H':
                LW=True
            elif res=='D':
                D=True
            else:
                LL=True
        else:
            H=df.iloc[i]['HomeTeam']
            L=df.iloc[i]['AwayTeam']
            if res=='H':
                LL=True
            elif res=='D':
                D=True
            else:
                LW=True
        
        if vars()[L][1]<x:
            l_points+=[0]
        else:
            l_points+=[sum(vars()[L][0][-x:])]
            
        if vars()[H][1]<x:
            h_points+=[0]
        else:
            h_points+=[sum(vars()[H][0][-x:])]
            
        # #   #   #   # #trata da  diferença dos pontos l-h        
        if vars()[L][1]>=x and vars()[H][1]>=x:
            points_diff+=[(sum(vars()[L][0][-x:]))-(sum(vars()[H][0][-x:]))] 
        else:
            points_diff+=[0]
        #   #  #  #   #   #  #   #   #  #   #  #  #    #  #  
        
        if LW:
            vars()[L][0]+=[3]
            vars()[H][0]+=[0]
        elif D:
            vars()[L][0]+=[1]
            vars()[H][0]+=[1]
        elif LL:
            vars()[L][0]+=[0]
            vars()[H][0]+=[3]
        else:
            print('Erro')
            return 'Erro'
        
        vars()[L][1]+=1
        vars()[H][1]+=1
    
    
    if diff:
        output=pd.DataFrame({'Points_Diff'+str(x):points_diff})
    else:
        output=pd.DataFrame({'L_Points'+str(x):l_points,'H_Points'+str(x):h_points,'Points_Diff'+str(x):points_diff})
    return output
        

In [15]:
def s_st_g_c(df,clubs,x,diff=True):
    for club in clubs:
        vars()[club]=[[],[],[],[],0]
    L_list=[[],[],[],[]] #0s,1st,2g,3c
    H_list=[[],[],[],[]]
    diff_list=[[],[],[],[]]
    l_rel=[[],[],[],[]] #relação >< que a média Low (1, maior q a media/0, menor/igual à média )
    h_rel=[[],[],[],[]] #relação >< que a média High
    #df=pd.read_excel(file+'.xlsx',0)
    size=len(df.index)
    for i in range(size):
        if df.iloc[i]['Lowest_Odd']=='H':
            L=df.iloc[i]['HomeTeam']
            H=df.iloc[i]['AwayTeam']
        else:
            H=df.iloc[i]['HomeTeam']
            L=df.iloc[i]['AwayTeam']
            
        
        if vars()[L][-1]<x:
            L_list[0]+=[0]
            L_list[1]+=[0]
            L_list[2]+=[0]
            L_list[3]+=[0]
        else:
            L_list[0]+=[int(round(sum(vars()[L][0][-x:])/x))]
            if (sum(vars()[L][0][-x:])/x) < vars()[L][0][-1]:
                l_rel[0]+=[1]
            else:
                l_rel[0]+=[0]
                
            L_list[1]+=[int(round(sum(vars()[L][1][-x:])/x))]
            if (sum(vars()[L][1][-x:])/x) < vars()[L][1][-1]:
                l_rel[1]+=[1]
            else:
                l_rel[1]+=[0]
                
            L_list[2]+=[int(round(sum(vars()[L][2][-x:])/x))]
            if (sum(vars()[L][2][-x:])/x) < vars()[L][2][-1]:
                l_rel[2]+=[1]
            else:
                l_rel[2]+=[0]
                
            L_list[3]+=[int(round(sum(vars()[L][3][-x:])/x))]
            if (sum(vars()[L][3][-x:])/x) < vars()[L][3][-1]:
                l_rel[3]+=[1]
            else:
                l_rel[3]+=[0]
            
        if vars()[H][-1]<x:
            H_list[0]+=[0]
            H_list[1]+=[0]
            H_list[2]+=[0]
            H_list[3]+=[0]
        else:
            H_list[0]+=[int(round(sum(vars()[H][0][-x:]))/x)]
            if (sum(vars()[H][0][-x:])/x) < vars()[H][0][-1]:
                h_rel[0]+=[1]
            else:
                h_rel[0]+=[0]
                
            H_list[1]+=[int(round(sum(vars()[H][1][-x:]))/x)]
            if (sum(vars()[H][1][-x:])/x) < vars()[H][1][-1]:
                h_rel[1]+=[1]
            else:
                h_rel[1]+=[0]
                
            H_list[2]+=[int(round(sum(vars()[H][2][-x:])/x))]
            if (sum(vars()[H][2][-x:])/x) < vars()[H][2][-1]:
                h_rel[2]+=[1]
            else:
                h_rel[2]+=[0]
                
            H_list[3]+=[int(round(sum(vars()[H][3][-x:])/x))]
            if (sum(vars()[H][3][-x:])/x) < vars()[H][3][-1]:
                h_rel[3]+=[1]
            else:
                h_rel[3]+=[0]
                
        # trata da relação de diferença     #   #   #  #  #   #   #  #   #  #  #   #   
        if vars()[L][-1]>=x and vars()[H][-1]>=x:
            diff_list[0]+=[int(round((sum(vars()[L][0][-x:])/x)-(sum(vars()[H][0][-x:])/x)))]
            diff_list[1]+=[int(round((sum(vars()[L][1][-x:])/x)-(sum(vars()[H][1][-x:])/x)))]
            diff_list[2]+=[int(round((sum(vars()[L][2][-x:])/x)-(sum(vars()[H][2][-x:])/x)))]
            diff_list[3]+=[int(round((sum(vars()[L][3][-x:])/x)-(sum(vars()[H][3][-x:])/x)))]
        else:
            diff_list[0]+=[0]
            diff_list[1]+=[0]
            diff_list[2]+=[0]
            diff_list[3]+=[0]
        # # ###   ##     #     #   #  #  #   #   #  #   #  #   #  #  #  #   #  #  #     
            
        if df.iloc[i]['Lowest_Odd']=='H':
            vars()[L][0]+=[df.iloc[i]['HS']]
            vars()[H][0]+=[df.iloc[i]['AS']]
            vars()[L][1]+=[df.iloc[i]['HST']]
            vars()[H][1]+=[df.iloc[i]['AST']]
            vars()[L][2]+=[df.iloc[i]['FTHG']]
            vars()[H][2]+=[df.iloc[i]['FTAG']]
            vars()[L][3]+=[df.iloc[i]['FTAG']]
            vars()[H][3]+=[df.iloc[i]['FTHG']]
            
        else:
            vars()[L][0]+=[df.iloc[i]['AS']]
            vars()[H][0]+=[df.iloc[i]['HS']]
            vars()[L][1]+=[df.iloc[i]['AST']]
            vars()[H][1]+=[df.iloc[i]['HST']]
            vars()[L][2]+=[df.iloc[i]['FTAG']]
            vars()[H][2]+=[df.iloc[i]['FTHG']]
            vars()[L][3]+=[df.iloc[i]['FTHG']]
            vars()[H][3]+=[df.iloc[i]['FTAG']]
               
        vars()[L][-1]+=1
        vars()[H][-1]+=1
            
            
    if diff:
        output=pd.DataFrame({'S_Diff'+str(x):diff_list[0],'ST_Diff'+str(x):diff_list[1],'Scrd_Diff'+str(x):diff_list[2],'Concd_Diff'+str(x):diff_list[3]})
    else:
        output=pd.DataFrame({'LS_'+str(x):L_list[0],'Last_L_S':l_rel[0],'LST_'+str(x):L_list[1],'Last_L_ST':l_rel[1],
                             'LScrd_'+str(x):L_list[2],'Last_L_Scrd':l_rel[2],'LSConcd_'+str(x):L_list[3],'Last_L_Concd':l_rel[3],
                            'HS_'+str(x):H_list[0],'Last_H_S':h_rel[0],'HST_'+str(x):H_list[1],'Last_H_ST':h_rel[1],
                             'HScrd_'+str(x):H_list[2],'Last_H_Scrd':h_rel[2],'HConcd_'+str(x):H_list[3],'Last_H_Concd':h_rel[3],
                            'S_Diff'+str(x):diff_list[0],'ST_Diff'+str(x):diff_list[1],
                             'Scrd_Diff'+str(x):diff_list[2],'Concd_Diff'+str(x):diff_list[3]})
            
    return output 

In [16]:
def non_odd_counter(df,clubs):
    for club in clubs:
        vars()[club]=[0,0,0,0] #-1 counter ||| 0 Win, 1 Scrd, 2 Concd
    L_count=[[],[],[]]
    H_count=[[],[],[]]
        
    #df=pd.read_excel(file+'.xlsx',0)    
    size=len(df.index)
    for i in range(size):
        if df.iloc[i]['Lowest_Odd']=='H':
            L=df.iloc[i]['HomeTeam']
            H=df.iloc[i]['AwayTeam']
        else:
            H=df.iloc[i]['HomeTeam']
            L=df.iloc[i]['AwayTeam']
               
        L_count[0]+=[vars()[L][0]]
        L_count[1]+=[vars()[L][1]]
        L_count[2]+=[vars()[L][2]]
        
        H_count[0]+=[vars()[H][0]]
        H_count[1]+=[vars()[H][1]]
        H_count[2]+=[vars()[H][2]]
        
        
        if df.iloc[i]['Lowest_Odd']=='H':   #Odd + baixa CASA 
            if df.iloc[i]['FTR']=='H': #se a equipa de casa ganha, como fica o counter
                if vars()[L][0]>0:
                    vars()[L][0]+=1 
                else:
                    vars()[L][0]=1
                if vars()[H][0]>0:
                    vars()[H][0]=-1 
                else:
                    vars()[H][0]-=1 
            if df.iloc[i]['FTR']=='A': #se a equipa de fora ganha, como fica o counter
                if vars()[L][0]>0:
                    vars()[L][0]=-1 
                else:
                    vars()[L][0]-=1
                if vars()[H][0]>0:
                    vars()[H][0]+=1 
                else:
                    vars()[H][0]=1 
            if df.iloc[i]['FTR']=='D': #se há empate, os counter de jogos ganhos vão todos a zero
                if vars()[L][0]>0:
                    vars()[L][0]=-1 
                else:
                    vars()[L][0]-=1
                if vars()[H][0]>0:
                    vars()[H][0]=-1 
                else:
                    vars()[H][0]-=1 
            if df.iloc[i]['FTHG']>0 and df.iloc[i]['FTAG']>0: # se ambas marcam
                #print('(+,+)')
                if vars()[L][1]>0:
                    vars()[L][1]+=1 
                else:
                    vars()[L][1]=1    
                if vars()[H][1]>0:
                    vars()[H][1]+=1    
                else:
                    vars()[H][1]=1    
                if vars()[L][2]>0:
                    vars()[L][2]=-1
                else:
                    vars()[L][2]-=1
                if vars()[H][2]>0:
                    vars()[H][2]=-1 
                else:
                    vars()[H][2]-=1 
            if df.iloc[i]['FTHG']>0 and df.iloc[i]['FTAG']==0: # se só casa marca
                #print('(+,0)')
                if vars()[L][1]>0:
                    vars()[L][1]+=1 
                else:
                    vars()[L][1]=1    
                if vars()[H][1]>0:
                    vars()[H][1]=-1    
                else:
                    vars()[H][1]-=1    
                if vars()[L][2]>0:
                    vars()[L][2]+=1
                else:
                    vars()[L][2]=1
                if vars()[H][2]>0:
                    vars()[H][2]=-1 
                else:
                    vars()[H][2]-=1 
            if df.iloc[i]['FTHG']==0 and df.iloc[i]['FTAG']>0: # se só fora marca
                #print('(0,+)')
                if vars()[L][1]>0:
                    vars()[L][1]=-1 
                else:
                    vars()[L][1]-=1    
                if vars()[H][1]>0:
                    vars()[H][1]+=1    
                else:
                    vars()[H][1]=1    
                if vars()[L][2]>0:
                    vars()[L][2]=-1
                else:
                    vars()[L][2]-=1
                if vars()[H][2]>0:
                    vars()[H][2]+=1 
                else:
                    vars()[H][2]=1 
            if df.iloc[i]['FTHG']==0 and df.iloc[i]['FTAG']==0: # se nenhuma marca
                #print('(0,0)')
                if vars()[L][1]>0:
                    vars()[L][1]=-1 
                else:
                    vars()[L][1]-=1    
                if vars()[H][1]>0:
                    vars()[H][1]=-1    
                else:
                    vars()[H][1]-=1    
                if vars()[L][2]>0:
                    vars()[L][2]+=1
                else:
                    vars()[L][2]=1
                if vars()[H][2]>0:
                    vars()[H][2]+=1 
                else:
                    vars()[H][2]=1 
        
        if df.iloc[i]['Lowest_Odd']=='A': 
            if df.iloc[i]['FTR']=='H': #se a equipa de casa ganha, como fica o counter
                if vars()[L][0]>0:
                    vars()[L][0]=-1 
                else:
                    vars()[L][0]-=1
                if vars()[H][0]>0:
                    vars()[H][0]+=1 
                else:
                    vars()[H][0]=1 
            if df.iloc[i]['FTR']=='A': #se a equipa de fora ganha, como fica o counter
                if vars()[L][0]>0:
                    vars()[L][0]+=1 
                else:
                    vars()[L][0]=1
                if vars()[H][0]>0:
                    vars()[H][0]=-1 
                else:
                    vars()[H][0]-=1 
            if df.iloc[i]['FTR']=='D': #se há empate, os counter de jogos ganhos vão todos a negativo
                if vars()[L][0]>0:
                    vars()[L][0]=-1 
                else:
                    vars()[L][0]-=1
                if vars()[H][0]>0:
                    vars()[H][0]=-1 
                else:
                    vars()[H][0]-=1 
            if df.iloc[i]['FTHG']>0 and df.iloc[i]['FTAG']>0: # se ambas marcam
                #print('(+,+)')
                if vars()[L][1]>0:
                    vars()[L][1]+=1 
                else:
                    vars()[L][1]=1    
                if vars()[H][1]>0:
                    vars()[H][1]+=1    
                else:
                    vars()[H][1]=1    
                if vars()[L][2]>0:
                    vars()[L][2]=-1
                else:
                    vars()[L][2]-=1
                if vars()[H][2]>0:
                    vars()[H][2]=-1 
                else:
                    vars()[H][2]-=1 
            if df.iloc[i]['FTHG']>0 and df.iloc[i]['FTAG']==0: # se só casa marca
                #print('(+,0)')
                if vars()[L][1]>0:
                    vars()[L][1]=-1 
                else:
                    vars()[L][1]-=1    
                if vars()[H][1]>0:
                    vars()[H][1]+=1    
                else:
                    vars()[H][1]=1    
                if vars()[L][2]>0:
                    vars()[L][2]=-1
                else:
                    vars()[L][2]-=1
                if vars()[H][2]>0:
                    vars()[H][2]+=1 
                else:
                    vars()[H][2]=1 
            if df.iloc[i]['FTHG']==0 and df.iloc[i]['FTAG']>0: # se só fora marca
                #print('(0,+)')
                if vars()[L][1]>0:
                    vars()[L][1]+=1 
                else:
                    vars()[L][1]=1    
                if vars()[H][1]>0:
                    vars()[H][1]=-1    
                else:
                    vars()[H][1]-=1    
                if vars()[L][2]>0:
                    vars()[L][2]+=1
                else:
                    vars()[L][2]=1
                if vars()[H][2]>0:
                    vars()[H][2]=-1 
                else:
                    vars()[H][2]-=1 
            if df.iloc[i]['FTHG']==0 and df.iloc[i]['FTAG']==0: # se nenhuma marca
                #print('(0,0)')
                if vars()[L][1]>0:
                    vars()[L][1]=-1 
                else:
                    vars()[L][1]-=1    
                if vars()[H][1]>0:
                    vars()[H][1]=-1    
                else:
                    vars()[H][1]-=1    
                if vars()[L][2]>0:
                    vars()[L][2]+=1
                else:
                    vars()[L][2]=1
                if vars()[H][2]>0:
                    vars()[H][2]+=1 
                else:
                    vars()[H][2]=1 
                    
    output=pd.DataFrame({'WinCounter_L':L_count[0],'WinCounter_H':H_count[0],'ScrdCounter_L':L_count[1],
                         'ScrdCounter_H':H_count[1],'ConcdCounter_L':L_count[2],'ConcdCounter_H':H_count[2]})
    return output

In [17]:
def odd_counter(df):
    odds=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] #odd counter (já vai ter counter negativos) (1,2.8,0.1)
    oddsput=[] #põe em coluna
    #df=pd.read_excel(file+'.xlsx',0)
    size=len(df.index)
    for i in range(size):
        if df.iloc[i]['Lowest_Odd']=='H':
            oddzz=round((df.iloc[i]['ODDH_Aver.'])*100)
            oddsput+=[odds[int((oddzz-100)//10)]]
        else:
            oddzz=round((df.iloc[i]['ODDA_Aver.'])*100)
            oddsput+=[odds[int((oddzz-100)//10)]]
            
        #counter
        if df.iloc[i]['Lowest_Odd']=='H':
            if df.iloc[i]['FTR']=='H': 
                if odds[int((oddzz-100)//10)]>0:
                    odds[int((oddzz-100)//10)]+=1
                else:
                    odds[int((oddzz-100)//10)]=1
            else: #se a equipa da casa odd + baixa não ganhar
                if odds[int((oddzz-100)//10)]>0:
                    odds[int((oddzz-100)//10)]=-1
                else:
                    odds[int((oddzz-100)//10)]-=1

        else: # se a odd mais baixa for a equipa de fora
            if df.iloc[i]['FTR']=='H':
                if odds[int((oddzz-100)//10)]>0:
                    odds[int((oddzz-100)//10)]=-1
                else:
                    odds[int((oddzz-100)//10)]-=1
            else: #se a equipa de fora (odd + baixa ) ganhar
                if odds[int((oddzz-100)//10)]>0:
                    odds[int((oddzz-100)//10)]+=1
                else:
                    odds[int((oddzz-100)//10)]=1
                
    output=pd.DataFrame({'LeagueOdd_Counter':oddsput})            
    return output

In [18]:
def lowest_lost(df,clubs):
    for club in clubs:
        vars()[club]=1000
    llost=[]
    size=len(df.index)
    for i in range(size):
        H=False
        A=False
        if df.iloc[i]['Lowest_Odd']=='H':
            L=df.iloc[i]['HomeTeam']
            Hi=df.iloc[i]['AwayTeam'] #high odd
            H=True
        else:
            L=df.iloc[i]['AwayTeam']
            Hi=df.iloc[i]['HomeTeam'] #high odd
            A=True
            
        llost+=[round_down(vars()[L],1)] #uso o round_down(,1)
        
        if H and (df.iloc[i]['FTR']!='H'):
            if df.iloc[i]['ODDH_Aver.']<vars()[L]:
                vars()[L]=df.iloc[i]['ODDH_Aver.']
        elif A and (df.iloc[i]['FTR']!='A'):
            if df.iloc[i]['ODDA_Aver.']<vars()[L]:
                vars()[L]=df.iloc[i]['ODDA_Aver.']
                
        if H and (df.iloc[i]['FTR']!='A'):
            if df.iloc[i]['ODDA_Aver.']<vars()[Hi]:
                vars()[Hi]=df.iloc[i]['ODDA_Aver.']
        elif A and (df.iloc[i]['FTR']!='H'):
            if df.iloc[i]['ODDH_Aver.']<vars()[Hi]:
                vars()[Hi]=df.iloc[i]['ODDH_Aver.']
                
    output=pd.DataFrame({'L_LowestLost':llost})
    return output

In [19]:
def won_notwon(df):
    size=len(df.index)
    wnw=[]
    for i in range(size):
        H=False
        A=False
        if df.iloc[i]['Lowest_Odd']=='H':
            #L=df.iloc[i]['HomeTeam']
            #Hi=df.iloc[i]['AwayTeam'] #high odd
            H=True
        else:
           # L=df.iloc[i]['AwayTeam']
            #Hi=df.iloc[i]['HomeTeam'] #high odd
            A=True
        
        if H:
            if df.iloc[i]['FTR']=='H':
                wnw+=[1] #ganhou
            else:
                wnw+=[0]
        if A:
            if df.iloc[i]['FTR']=='A':
                wnw+=[1] #ganhou
            else:
                wnw+=[0]
                
    output=pd.DataFrame({'Won/NotWon':wnw})
    return output

In [20]:
def join_alpha(df,clubs,x,diff=True):
    #df=pd.read_excel(path+'.xlsx',0)
    a=low_draw_high(df)
    b=points_made(df,clubs,x,diff=diff)
    c=lastgames_stats(df,clubs,x,diff=diff)
    d=lowest_lost(df,clubs)
    e=s_st_g_c(df,clubs,x,diff=diff)
    f=non_odd_counter(df,clubs)
    g=odd_counter(df)
    h=won_notwon(df)
    df1=pd.concat([a,b,c,d,e,f,g,h],sort=False,axis=1)
    return df1

### odd_allcounter é para fazer no fim (qnd tivermos as ligas tds juntas)

In [21]:
def odd_allcounter_alpha(file):  #faz o count das odds qnd já juntámos várias ligas
    #Aqui não precisamos do ficheiro original, pq este já tem as odds
    
    limit=len(file.index)
    odds=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] #odd counter (já vai ter counter negativos) (1,2.8,10)
    oddsput=[] #põe em coluna
    #for club in clubs:                 
    #    vars()[club]=[0] #agr só precisamos do counter do clube
    for i in range(limit):  
        oddzz = round((file.iloc[i]['LowOdd'])*100)
        #põe os counter na lista para adicionar
        #print(i,'índice do ficheiro')
        #print(file.iloc[i]['LowODD'],'odd')
        #print(int((oddzz-100)//10),'indíce da lista')
        #print(oddzz,'odd*100')
        oddsput+=[odds[int((oddzz-100)//10)]]    

        #counter
        if file.iloc[i]['Won/NotWon']==1: 
            if odds[int((oddzz-100)//10)]>0:
                odds[int((oddzz-100)//10)]+=1
            else:
                odds[int((oddzz-100)//10)]=1
        else: #se a equipa da odd + baixa não ganhar
            if odds[int((oddzz-100)//10)]>0:
                odds[int((oddzz-100)//10)]=-1
            else:
                odds[int((oddzz-100)//10)]-=1
    
    print(len(file.index),len(oddsput))
    output=pd.DataFrame({'ALLOdd_Counter':oddsput})
    file=pd.concat([file,output],sort=False,axis=1)
    return file    

In [22]:
def alpha_concat(files): #faz concat da lista de files e ordena-os já por date e time
    df1=pd.concat(files,sort=False)
    df=df1.sort_values(['Date'], ascending=True)
    return df

Este `cut_zeros` só serve para criar os ficheiros que juntam épocas de várias ligas  

In [23]:
def cut_zeros(file,x): #retira os dados com counters/médias/pontos a zero
    #precisamos de indicar o x dos jogos para a média
    size=len(file.index) #vamos só usar uma coluna, cuidado com erros
    i=0
    done=False
    conter=0
    ind=[]
    while i<size and not(done):
        if not((file.iloc[i]['Points_Diff'+str(x)]==0) and (file.iloc[i]['S_Diff'+str(x)]==0) and (file.iloc[i]['ST_Diff'+str(x)]==0)):
            ind+=[i]
            counter+=1
        else:
            counter=0
        i+=1
        if counter==100: #se encontrarmos 100 jogos seguidos sem zeros, acaba o cicl0
            done=True
    conc=[]
    for n in ind:
        conc+=[pd.DataFrame(file.iloc[n,:]).T]
    conc+=[file[i:]]
    df=pd.concat(conc,sort=False)
    return df

`cut_zeros2` é para só para ser usado numa época de uma só liga

In [24]:
def cut_zeros2(file,x):
    #precisamos de indicar o x dos jogos para a média
    size=len(file.index) #vamos só usar uma coluna, cuidado com erros
    i=0
    done=False
    conter=0
    ind=[]
    while i<size and not(done):
        if not((file.iloc[i]['Points_Diff'+str(x)]==0) and (file.iloc[i]['S_Diff'+str(x)]==0) and (file.iloc[i]['ST_Diff'+str(x)]==0)):
            ind+=[i]
            counter+=1
        else:
            counter=0
        i+=1
        if counter==25: #se encontrarmos 100 jogos seguidos sem zeros, acaba o cicl0
            done=True
    conc=[]
    for n in ind:
        conc+=[pd.DataFrame(file.iloc[n,:]).T]
    conc+=[file[i:]]
    df=pd.concat(conc,sort=False)
    return df

### byodd_alpha 
divide um ficheiro com folhas de odds

In [25]:
def byodd_alpha(path):
    file=pd.read_excel(path+'.xlsx',0)
    q=[[file]]
    for i in range(10,29,1):
        odd1=round(i/10,2)
        odd2=round(odd1+0.09,2)
        new=file.loc[(odd1<=file['LowOdd']) & (file['LowOdd']<=odd2)]
        q+=[[new,odd1,odd2]]
    with pd.ExcelWriter(path+".xlsx") as writer:
        for n, df in enumerate(q):
            if n==0:
                df[0].to_excel(writer, 'ALPHA_'+path, index=None)
            else:
                df[0].to_excel(writer, "%d_ODD_%s" %(df[1],df[2]) +"_"+path, index=None)
    writer.save()

In [26]:
def alpha_auxfile(path,clubs,x,diff=True): #trata de um ficheiro para o ALPHA
    
    file=all_datetime(path,write=False) #põe as datas em datetime (ficheiro original)
    file=join_alpha(file,clubs,x,diff=diff) #transforma o ficheiro original no ficheiro para ALPHA
    
    return file

`year_alpha` junta num só ficheiro tds as ligas de uma época. 

In [27]:
def year_alpha(os_path,x,season,diff=True): 
    #junta num só ficheiro tds as ligas de uma época
    import os
    ind=find_data(os_path)
    rep=[] #repositorio para fazer concat
    for data_file in ind:
        season_list=os.listdir(os_path+data_file+'\\')
        i=fileyear_to_ind(season)
        print(os_path+data_file+'\\'+season_list[i][:-5])
        clubs=findall_clubs2(os_path+data_file+'\\'+season_list[i][:-5])
        file=alpha_auxfile(os_path+data_file+'\\'+season_list[i][:-5],clubs,x,diff=diff)
        rep+=[file]
    df=pd.concat(rep,sort=False)
    df=df.sort_values(['Date'],ascending=True)
    df=df.reset_index(drop=True)
    return df

`league_alpha` junta num só ficheiro as épocas que queremos de uma liga

In [28]:
def league_alpha(path,f_season,l_season,x,diff=True):
    os_path='C:\\Users\\joaom\\Documents\\Projetos\\PYTHON\\Apostas\\'+path+'___\\'
    i1=fileyear_to_ind(f_season)
    i2=fileyear_to_ind(l_season)
    seasons=os.listdir(os_path)
    rep=[]
    for i in range(i1,i2+1):
        clubs=findall_clubs2(os_path+seasons[i][:-5])
        file=alpha_auxfile(os_path+seasons[i][:-5],clubs,x,diff=diff)
        file=cut_zeros2(file,x)
        rep+=[file]
    df=pd.concat(rep,sort=False)
    df=df.sort_values(['Date'],ascending=True)
    df=df.reset_index(drop=True)
    return df
    

`year_complete` junta o AllOddCounter, e faz o cut_zeros (e também põe o Won/NotWon na última coluna).

Recebe o file de `year_alpha` 

Como não temos Time, os jogos não estão bem por ordem (entre ligas), por iss, nesses casos não vamos usar all_odd_counter

In [29]:
def year_complete(file,x,odd_allcounter=False):
    if odd_allcounter:
        file=odd_allcounter_alpha(file)
        colunas=list(file.columns.values)
        file=file[colunas[:-2]+[colunas[-1]]+[colunas[-2]]]
        file=cut_zeros(file,x)
        file=file.reset_index(drop=True)
    else:
        file=cut_zeros(file,x)
        file=file.reset_index(drop=True)
    return file

`all_alpha` prepara o ficheiro para passar para as pastas do PC e faz o `year_alpha` para os anos que queremos

In [30]:
def all_alpha(os_path,x,f_season,l_season,diff=True,odd_allcounter=False):
    rep=[]
    for i in range(f_season,l_season+1):
        df=year_alpha(os_path,x,i,diff=diff)
        df=year_complete(df,x,odd_allcounter=odd_allcounter)
        rep+=[df]
    bigfile=pd.concat(rep,sort=False)
    bigfile=bigfile.sort_values(['Date'],ascending=True)
    bigfile=bigfile.reset_index(drop=True)
    return bigfile

In [32]:
#BigAlpha18_20=all_alpha('C:\\Users\\joaom\\Documents\\Projetos\\PYTHON\\Apostas\\',5,2018,2020)
#BigAlpha18_20

In [33]:
BigAlpha18_20.to_excel('BigAlpha18_20.xlsx',index=False)

NameError: name 'BigAlpha18_20' is not defined

In [ ]:
byodd_alpha('BigAlpha18_20')

____________________

### Let the tests begin

In [ ]:
#year_alpha('C:\\Users\\joaom\\Documents\\Projetos\\PYTHON\\Apostas\\',5,2019)

In [ ]:
#sqe=year_alpha('C:\\Users\\joaom\\Documents\\Projetos\\PYTHON\\Apostas\\',5,2019)

In [ ]:
#year_complete(sqe,5,odd_allcounter=True)

In [44]:
#sqe

In [72]:
PremAlpha10_20=league_alpha('Premier_League',2006,2020,5)

In [74]:
PremAlpha10_20.to_excel('PremAlpha06_20.xlsx',index=False)

In [75]:
byodd_alpha('PremAlpha06_20')

In [64]:
PremAlpha15_20=league_alpha('Premier_League',2015,2020,5)
PremAlpha18_20=league_alpha('Premier_League',2018,2020,5)
PremAlpha20=league_alpha('Premier_League',2020,2020,5)

In [66]:
PremAlpha15_20.to_excel('PremAlpha15_20.xlsx',index=False)
PremAlpha18_20.to_excel('PremAlpha18_20.xlsx',index=False)
PremAlpha20.to_excel('PremAlpha20.xlsx',index=False)

In [67]:
byodd_alpha('PremAlpha15_20')
byodd_alpha('PremAlpha18_20')
byodd_alpha('PremAlpha20')

In [31]:
PremAlpha06_19=league_alpha('Premier_League',2006,2019,5)
PremAlpha06_18=league_alpha('Premier_League',2006,2018,5)
PremAlpha06_17=league_alpha('Premier_League',2006,2017,5)

In [32]:
PremAlpha06_19.to_excel('PremAlpha06_19.xlsx',index=False)
PremAlpha06_18.to_excel('PremAlpha06_18.xlsx',index=False)
PremAlpha06_17.to_excel('PremAlpha06_17.xlsx',index=False)

In [33]:
byodd_alpha('PremAlpha06_19')
byodd_alpha('PremAlpha06_18')
byodd_alpha('PremAlpha06_17')

In [34]:
PremAlpha18=league_alpha('Premier_League',2018,2018,5)
PremAlpha19=league_alpha('Premier_League',2019,2019,5)

In [35]:
PremAlpha18.to_excel('PremAlpha18.xlsx',index=False)
PremAlpha19.to_excel('PremAlpha19.xlsx',index=False)

In [36]:
byodd_alpha('PremAlpha18')
byodd_alpha('PremAlpha19')